In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rhea.db.zinc20 import Zinc20
from rhea.chem.molecule import Molecule
from titan.llm.tools import PromptStructure

In [ ]:
samples = 30_000
num_transforms = 5
stop = False
eos = '<eos>'

with Zinc20() as db:
    c = 0
    for mol in db.iter_mols():
       
        mol = Chem.AddHs(mol)
        # TO 3D1
        AllChem.EmbedMolecule(mol)
        try:
            AllChem.MMFFOptimizeMolecule(mol)
        except ValueError:
            print('Failed to optimize molecule', end='\r')
            continue
        
        try:
            Chem.SanitizeMol(mol)   
            m = Molecule(mol=mol)
            no_hs = Chem.RemoveHs(m.molecule)
        except ValueError:
            continue
        
        for _ in range(num_transforms):    
            m = Molecule(mol=no_hs)
            m.transform_random()                        # random transformation (rot, trans)
            converter = PromptStructure()
            p = converter.to_prompt(m)                  # <MOL>smile\n<XYZ>coordinates
            # append to file
            with open(f'./data/xyz_mols/input.txt', 'a') as f:
                f.write(p)
                f.write('\n')
            new_mol = converter.str_to_molecule(p)
            
            c += 1
            print(c, end='\r')
            
        if c == samples and stop:
            break        